# PV Degradation Tools Tutorial-1
### EnergyCalcs and RelativeHumidity
***
Matthew Brown

2022.12.29
***

**Requirements:**
- weather file (psm3 preferred) demo is provided
- python package `pvlib`

**Objectives:**
1. Read in necessary weather data
2. Use `pvlib` to generate cell temp, module temp, and POA
3. Calculate some water ingress parameters
4. Calculate VantHoff degradation acceleration factor
5. Calculate Arrhenius degradation acceleration factor
6. Calculate various material humidities
7. Solder Fatigue

**Miscillaneous:**

For help with `pvlib`, see https://pvlib-python.readthedocs.io/en/stable/index.html

In [1]:
import pvlib
import numpy as np
import pandas as pd

import pvdeg 

### 1.
From a psm3 file we need the following:
- DNI
- DHI
- GHI
- Cloud Cover
- Temperature, Dry-Bulb
- Temperature, Dew-Point
- Wind Speed
- Relative Humidity
- Latitude
- Longitude

In [3]:
sam_file = r'..\..\tests\data\psm3_pytest.csv'
data, meta = pvlib.iotools.read_psm3(filename=sam_file)

dni = data.DNI
dhi = data.DHI
ghi = data.GHI

temp_dry = data.Temperature

time_range = data.index

### 2.

We will use `pvlib` to generate data necessary for pvdeg Tools. For help with these functions, see https://pvlib-python.readthedocs.io/en/stable/index.html

In [4]:
solar_pos = pvlib.solarposition.get_solarposition(time=time_range, latitude=meta['Latitude'],
                                            longitude=meta['Longitude'], altitude=meta['Elevation'])

poa_df = pvlib.irradiance.get_total_irradiance(surface_tilt=20, surface_azimuth=180, 
                                                    solar_zenith=solar_pos.apparent_zenith,
                                                    solar_azimuth=solar_pos.azimuth,
                                                    dni=dni, ghi=ghi, dhi=dhi)
poa_global = poa_df.poa_global

### 3. Water Vapor Pressure & Edge Seal Width

From here on, pvdeg Tools 

In [7]:
water_vapor_pressure = pvdeg.StressFactors.water_vapor_pressure(data['Dew Point'])

avg_wvp = water_vapor_pressure.mean()
k = pvdeg.StressFactors.k(avg_wvp=avg_wvp)

edge_seal_width = pvdeg.StressFactors.edge_seal_width(k)

print(avg_wvp)
print(k)
print(edge_seal_width)

# Currently No Data: Cloud Cover
# dew_yield = pvdeg.EnergyCalcs.dew_yield(elevation=meta['Elevation'], dew_point=data['Dew Point']
#                                         dry_bulb=data['Temperature'],
#                                         wind_speed=data['Wind Speed'],
#                                         n=)

0.542181628310062
0.0009611628598096772
0.4499535870958669


### 4. VantHoff Degradation

In [9]:
# chamber irradiance (W/m^2)
I_chamber = 1000 
# chamber temperature (C)
temp_chamber = 60

# calculate the cell temperature with pvlib (see documentation for help)
cell_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']

temp_cell = pvlib.temperature.sapm_cell(poa_global=poa_global, temp_air=data['Temperature'],
                                        wind_speed=data['Wind Speed'], **cell_params)

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.Degradation.vantHoff_deg(I_chamber=I_chamber, poa_global=poa_global,
                                            temp_cell=temp_cell, temp_chamber=temp_chamber)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.Degradation.IwaVantHoff(poa_global=poa_global, temp_cell=temp_cell)

In [10]:
print(vantHoff_deg)
print(irr_weighted_avg_v)

8.383035313690966
232.47810387760708


### 5. Arrhenius Degradation

In [13]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

# calculate the module temperature with pvlib (see documentation for help)
temp_module = pvlib.temperature.sapm_module(poa_global=poa_global, temp_air=data['Temperature'],
                                            wind_speed=data['Wind Speed'], a=cell_params['a'],
                                            b=cell_params['b'])

rh_surface = pvdeg.StressFactors.rh_surface_outside(rh_ambient=data['Relative Humidity'],
                                                        temp_ambient=data['Temperature'],
                                                        temp_module=temp_module)

arrhenius_deg = pvdeg.Degradation.arrhenius_deg(I_chamber=I_chamber, rh_chamber=rh_chamber,
                                              rh_outdoor=rh_surface, poa_global=poa_global,
                                              temp_chamber=temp_chamber, temp_cell=temp_cell,
                                              Ea=Ea)

irr_weighted_avg_a = pvdeg.Degradation.IwaArrhenius(poa_global=poa_global, 
                                                  rh_outdoor=data['Relative Humidity'],
                                                  temp_cell=temp_cell, Ea=Ea)

In [14]:
print(rh_surface[17])
print(arrhenius_deg)
print(irr_weighted_avg_a)

81.99
12.725956447979469
194.6613637632573


### 6. Material Relative Humidity

In [18]:
rh_front_encap = pvdeg.StressFactors.rh_front_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_back_encap = pvdeg.StressFactors.rh_back_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_backsheet = pvdeg.StressFactors.rh_backsheet_from_encap(rh_back_encap=rh_back_encap,
                                                         rh_surface_outside=data['Relative Humidity'])

In [19]:
print(rh_front_encap[17])
print(rh_back_encap[17])
print(rh_backsheet[17])


50.28910527924019
80.45763598088315
81.22381799044157
